In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/quora-question-pairs/test.csv.zip
/kaggle/input/quora-question-pairs/test.csv
/kaggle/input/quora-question-pairs/train.csv.zip
/kaggle/input/quora-question-pairs/sample_submission.csv.zip


Import necessary modules.

In [ ]:
pip install transformers

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [11]:
transformers.__version__

'3.0.2'

Load data.

In [3]:
df = pd.read_csv('../input/quora-question-pairs/train.csv.zip') # use 100k samples for training
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [4]:
df.describe()

,id,qid1,qid2,is_duplicate
count,404290.000000,404290.000000,404290.000000,404290.000000
mean,202144.500000,217243.942418,220955.655337,0.369198
std,116708.614502,157751.700002,159903.182629,0.482588
min,0.000000,1.000000,2.000000,0.000000
25%,101072.250000,74437.500000,74727.000000,0.000000
50%,202144.500000,192182.000000,197052.000000,0.000000
75%,303216.750000,346573.500000,354692.500000,1.000000
max,404289.000000,537932.000000,537933.000000,1.000000


In [5]:
df.isnull().sum()

id              0
qid1            0
qid2            0
question1       1
question2       2
is_duplicate    0
dtype: int64

In [6]:
df.dropna(axis=0, inplace=True)
df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [7]:
measurer = np.vectorize(len)
dict(zip(df, measurer(df.values.astype(str)).max(axis=0)))

{'id': 6,
 'qid1': 6,
 'qid2': 6,
 'question1': 623,
 'question2': 1169,
 'is_duplicate': 1}

Let's split our dataset into training and validation sets.

In [8]:
len(df)

404287

In [9]:
train_df, val_df, test_df = df.loc[:0.7 * len(df)], df.loc[0.7 * len(df):0.9 * len(df)], df.loc[0.9 * len(df):]

In [10]:
train_df.shape

(282999, 6)

In [11]:
val_df.shape

(80857, 6)

In [12]:
test_df.shape

(40431, 6)

In [13]:
print("Train Target Distribution")
print(train_df.is_duplicate.value_counts())

Train Target Distribution
0    177877
1    105122
Name: is_duplicate, dtype: int64


In [14]:
print("Validation Target Distribution")
print(val_df.is_duplicate.value_counts())

Validation Target Distribution
0    50805
1    30052
Name: is_duplicate, dtype: int64


In [15]:
print("Test Target Distribution")
print(test_df.is_duplicate.value_counts())

Test Target Distribution
0    26342
1    14089
Name: is_duplicate, dtype: int64


In [16]:
y_train = tf.keras.utils.to_categorical(train_df.is_duplicate, num_classes=2)

y_val = tf.keras.utils.to_categorical(val_df.is_duplicate, num_classes=2)

y_test = tf.keras.utils.to_categorical(test_df.is_duplicate, num_classes=2)

In [17]:
y_train.shape

(282999, 2)

Custom data generator used from keras example.

In [18]:
max_length = 128  # Maximum length of input sentence to the model.
batch_size = 32
epochs = 2


labels = ['not duplicate', 'duplicate']

In [19]:
class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data.

    Args:
        sentence_pairs: Array of premise and hypothesis input sentences.
        labels: Array of labels.
        batch_size: Integer batch size.
        shuffle: boolean, whether to shuffle the data.
        include_targets: boolean, whether to incude the labels.

    Returns:
        Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
        (or just `[input_ids, attention_mask, `token_type_ids]`
         if `include_targets=False`)
    """

    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-base-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)

In [20]:
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Encoded token ids from BERT tokenizer.
    input_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="input_ids"
    )
    # Attention masks indicates to the model which tokens should be attended to.
    attention_masks = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="attention_masks"
    )
    # Token type ids are binary masks identifying different sequences in the model.
    token_type_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="token_type_ids"
    )
    # Loading pretrained BERT model.
    bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased")
    # Freeze the BERT model to reuse the pretrained features without modifying them.
    bert_model.trainable = False

    sequence_output, pooled_output = bert_model(
        input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
    )
    # Add trainable layers on top of frozen layers to adapt the pretrained features on the new data.
    bi_lstm = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, return_sequences=True)
    )(sequence_output)
    # Applying hybrid pooling approach to bi_lstm sequence output.
    avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
    max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)
    concat = tf.keras.layers.concatenate([avg_pool, max_pool])
    dropout = tf.keras.layers.Dropout(0.3)(concat)
    output = tf.keras.layers.Dense(2, activation="softmax")(dropout)
    model = tf.keras.models.Model(
        inputs=[input_ids, attention_masks, token_type_ids], outputs=output
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="categorical_crossentropy",
        metrics=["acc"],
    )


In [21]:
print(f"Strategy: {strategy}")
model.summary()

Strategy: <tensorflow.python.distribute.mirrored_strategy.MirroredStrategy object at 0x7f0e25b141d0>
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
attention_masks (InputLayer)    [(None, 128)]        0                                            
__________________________________________________________________________________________________
token_type_ids (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     ((None, 128, 768), ( 109482240   input_ids[0][0]     

In [22]:
train_data = BertSemanticDataGenerator(
    train_df[["question1", "question2"]].values.astype("str"),
    y_train,
    batch_size=batch_size,
    shuffle=True,
)
valid_data = BertSemanticDataGenerator(
    val_df[["question1", "question2"]].values.astype("str"),
    y_val,
    batch_size=batch_size,
    shuffle=False,
)


In [23]:
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
)

Epoch 1/2
8843/8843 [==============================] - 2148s 243ms/step - loss: 0.3934 - acc: 0.8101 - val_loss: 0.3406 - val_acc: 0.8420
Epoch 2/2
8843/8843 [==============================] - 2138s 242ms/step - loss: 0.3458 - acc: 0.8398 - val_loss: 0.3192 - val_acc: 0.8539


The above model froze the pre-trained model in order to reuse the pretrained features without modifying them.

The next thing to do is to fine-tune the model by unfreezing the BERT model and retraining with a very low rate. This can deliver meaningful improvement by incrementally adapting the pretrained features to the new data.

In [24]:
# Unfreeze the bert_model.
bert_model.trainable = True
# Recompile the model to make the change effective.
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
attention_masks (InputLayer)    [(None, 128)]        0                                            
__________________________________________________________________________________________________
token_type_ids (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     ((None, 128, 768), ( 109482240   input_ids[0][0]                  
                                                                 attention_masks[0][0] 

In [26]:
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs
)

Epoch 1/2
8843/8843 [==============================] - 4692s 531ms/step - loss: 0.2779 - accuracy: 0.8786 - val_loss: 0.2553 - val_accuracy: 0.8914
Epoch 2/2
8843/8843 [==============================] - 4688s 530ms/step - loss: 0.2094 - accuracy: 0.9132 - val_loss: 0.2437 - val_accuracy: 0.8999


In [27]:
test_data = BertSemanticDataGenerator(
    test_df[["question1", "question2"]].values.astype("str"),
    y_test,
    batch_size=batch_size,
    shuffle=False,
)
model.evaluate(test_data, verbose=1)

1263/1263 [==============================] - 221s 175ms/step - loss: 0.2420 - accuracy: 0.8986


[0.24202945828437805, 0.8986292481422424]

We have achieved 89% accuracy on the test data. Let's try out some sentences:

In [30]:
def check_similarity(sentence1, sentence2):
    sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
    test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
    )

    proba = model.predict(test_data)[0]
    idx = np.argmax(proba)
    proba = f"{proba[idx] * 100: .2f}%"
    pred = labels[idx]
    return pred, proba

In [31]:
q1 = "How old is Mark??"
q2 = "What is Mark's Age?"

check_similarity(q1, q2)

('duplicate', ' 85.29%')

In [34]:
model.save('Quora_duplicate_model', save_format='tf')

In [37]:
!zip -r 'Quora_duplicate_model.zip' './Quora_duplicate_model'

  adding: Quora_duplicate_model/ (stored 0%)
  adding: Quora_duplicate_model/assets/ (stored 0%)
  adding: Quora_duplicate_model/variables/ (stored 0%)
  adding: Quora_duplicate_model/variables/variables.data-00000-of-00001 (deflated 12%)
  adding: Quora_duplicate_model/variables/variables.index (deflated 80%)
  adding: Quora_duplicate_model/saved_model.pb (deflated 92%)


In [38]:
!zip -r 'Quora_duplicate_model_ZZ.zip' './Quora_duplicate_model'

  adding: Quora_duplicate_model/ (stored 0%)
  adding: Quora_duplicate_model/assets/ (stored 0%)
  adding: Quora_duplicate_model/variables/ (stored 0%)
  adding: Quora_duplicate_model/variables/variables.data-00000-of-00001 (deflated 12%)
  adding: Quora_duplicate_model/variables/variables.index (deflated 80%)
  adding: Quora_duplicate_model/saved_model.pb (deflated 92%)
